## DeFi Fees & Revenue Tracker On Plume Mainnet


### Step 1. Get Defillama TVL data On Plume And Create Dataframe

In [34]:
import requests
import pandas as pd

def get_plume_protocols_tvl():
    url = "https://api.llama.fi/protocols"
    response = requests.get(url)
    all_protocols = response.json()

    rows = []
    for p in all_protocols:
        if "Plume Mainnet" in p.get("chains", []) and p.get("chainTvls", {}).get("Plume Mainnet"):
            rows.append([
                p.get("name"),
                p.get("category", "Unknown"),
                p["chainTvls"]["Plume Mainnet"]
            ])

    df_protocols_tvl = pd.DataFrame(rows, columns=["Protocol", "Category", "TVL (USD)"])
    return df_protocols_tvl

if __name__ == "__main__":
    df_protocols_tvl = get_plume_protocols_tvl()
    print(df_protocols_tvl)

                  Protocol                   Category     TVL (USD)
0              Morpho Blue                    Lending  5.658061e+07
1                Curve DEX                       Dexs  8.836406e+06
2    DeSyn Liquid Strategy           Yield Aggregator  1.682465e+08
3                 Re7 Labs              Risk Curators  5.407904e+07
4             Pell Network                  Restaking  1.693605e+08
5                Midas RWA                        RWA  2.401717e+07
6          Superstate USTB                        RWA  2.446426e+07
7          Superstate USCC              Basis Trading  1.702485e+07
8             Origin Ether             Liquid Staking  8.140618e+04
9                  YieldFi           Yield Aggregator  1.098363e+07
10                 Nucleus           Yield Aggregator  5.775652e+07
11                 iZiSwap                       Dexs  4.147061e+06
12     DeSyn Basis Trading              Basis Trading  4.994855e+04
13             Nest Credit                      

### Step 2: Get DefiLlama Revenue Data On Plume And Create Dataframe

In [25]:
import requests
import pandas as pd

def get_plume_revenue():
    url = "https://api.llama.fi/overview/fees/Plume%20Mainnet?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyRevenue"
    response = requests.get(url)
    data = response.json()

    rows = []
    for p in data.get("protocols", []):
        rows.append([
            p.get("name"),
            p.get("total24h", 0),
            p.get("total7d", 0),
            p.get("total30d", 0),
            p.get("totalAllTime", 0)
        ])
    df_protocols_revenue = pd.DataFrame(rows, columns=["Protocol", "24h Revenue (USD)", "7d Revenue (USD)", "30d Revenue (USD)", "Total Revenue (USD)"])
    return df_protocols_revenue

if __name__ == "__main__":
    df_protocols_revenue = get_plume_revenue()
    print(df_protocols_revenue)

          Protocol  24h Revenue (USD)  7d Revenue (USD)  30d Revenue (USD)  \
0         deBridge                  0                 0                 95   
1      Satori Perp                  0                 2                105   
2          Ambient                  0                 0                  0   
3  Superstate USTB                107               873               2993   
4    EISEN Finance                  0                 0                  6   
5      ZNS Connect                  0                21                542   
6  Superstate USCC                350              2445              10270   

   Total Revenue (USD)  
0                 1441  
1                  452  
2                    0  
3                 2923  
4                   18  
5                24665  
6                19945  


### Step 3: Get DefiLlama Fees Data On Plume And Create Dataframe

In [26]:
import requests
import pandas as pd

def get_plume_fees():
    url = "https://api.llama.fi/overview/fees/Plume%20Mainnet?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true"
    response = requests.get(url)
    data = response.json()

    rows = []
    for p in data.get("protocols", []):
        rows.append([
            p.get("name"),
            p.get("total24h", 0),
            p.get("total7d", 0),
            p.get("total30d", 0),
            p.get("totalAllTime", 0)
        ])
    df_protocols_fees = pd.DataFrame(rows, columns=["Protocol", "24h Fees (USD)", "7d Fees (USD)", "30d Fees (USD)", "Total Fees (USD)"])
    return df_protocols_fees

if __name__ == "__main__":
    df_protocols_fees = get_plume_fees()
    print(df_protocols_fees)

           Protocol  24h Fees (USD)  7d Fees (USD)  30d Fees (USD)  \
0          deBridge               0              0              95   
1       Satori Perp               0              2             105   
2           Ambient             167            820            6358   
3   Superstate USTB            3066          24433           83826   
4     EISEN Finance               0              0               6   
5       ZNS Connect               0             21             542   
6   Superstate USCC            7430          57393          260330   
7  Rooster Protocol            1601          10840           71133   

   Total Fees (USD)  
0              1441  
1               452  
2             31092  
3             81894  
4                18  
5             24665  
6            558516  
7            203206  


### Step 4: Get CoinGecko Price and Market Cap Data

In [28]:
import requests
import pandas as pd
from tqdm import tqdm

def load_coingecko_list():
    url = 'https://api.coingecko.com/api/v3/coins/list'
    response = requests.get(url)
    return response.json() if response.status_code == 200 else []

def get_coingecko_id(protocol_name, coin_list):
    # Try exact name match, then symbol match
    protocol_name = protocol_name.lower()
    for coin in coin_list:
        if coin['name'].lower() == protocol_name:
            return coin['id']
    for coin in coin_list:
        if coin['symbol'].lower() == protocol_name:
            return coin['id']
    return None

def batch_fetch_market_data(coingecko_ids):
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        'vs_currency': 'usd',
        'ids': ','.join(coingecko_ids),
        'order': 'market_cap_desc',
        'per_page': len(coingecko_ids),
        'page': 1,
        'sparkline': 'false'
    }
    response = requests.get(url, params=params)
    return response.json() if response.status_code == 200 else []

def get_plume_protocols_market_data():
    # Get Plume protocols TVL DataFrame
    tvl_df = get_plume_protocols_tvl()
    coin_list = load_coingecko_list()
    protocols = tvl_df['Protocol'].tolist()
    
    # Step 1: Find CoinGecko IDs
    protocol_to_id = {}
    for protocol in tqdm(protocols, desc="Matching protocols to CoinGecko"):
        cg_id = get_coingecko_id(protocol, coin_list)
        if cg_id:
            protocol_to_id[protocol] = cg_id
    
    # Step 2: Fetch market data
    market_data = batch_fetch_market_data(list(protocol_to_id.values()))
    cg_data_map = {d['id']: d for d in market_data}
    
    # Step 3: Build DataFrame
    rows = []
    for protocol, cg_id in protocol_to_id.items():
        cg_data = cg_data_map.get(cg_id, {})
        rows.append([
            protocol,
            cg_data.get('symbol', '').upper(),
            cg_data.get('current_price', None),
            cg_data.get('market_cap', None)
        ])
    df = pd.DataFrame(rows, columns=['Protocol', 'Token Symbol', 'Current Price (USD)', 'Market Cap (USD)'])
    return df

if __name__ == "__main__":
    df_protocols_market_data = get_plume_protocols_market_data()
    print(df_protocols_market_data)

Matching protocols to CoinGecko: 100%|██████████| 36/36 [00:00<00:00, 244.01it/s]


          Protocol Token Symbol  Current Price (USD)  Market Cap (USD)
0  Superstate USCC         USCC            11.160000               0.0
1     Origin Ether         OETH          4300.240000       235018231.0
2         Credbull          CBL             0.000875          173225.0


### Use Joblib To Store Results

In [33]:
import joblib
import os

save_path = r"C:\Users\tobec\OneDrive\Documents\protocol_tracker\database"

os.makedirs(save_path, exist_ok=True)

joblib.dump(df_protocols_tvl, os.path.join(save_path, "df_protocols_tvl.joblib"))
joblib.dump(df_protocols_revenue, os.path.join(save_path, "df_protocols_revenue.joblib"))
joblib.dump(df_protocols_fees, os.path.join(save_path, "df_protocols_fees.joblib"))
joblib.dump(df_protocols_market_data, os.path.join(save_path, "df_protocols_market_data.joblib"))

['C:\\Users\\tobec\\OneDrive\\Documents\\protocol_tracker\\database\\df_protocols_market_data.joblib']